# Домашнее задание №1: Распознавание речи


In [ ]:
# Установка зависимостей
%pip install nvidia-ml-py
%pip install --upgrade jupyter ipywidgets

# Проверка установки nvidia-ml-py
try:
    import nvidia_ml_py3 as nvml
    print("✓ nvidia-ml-py успешно установлен")
except ImportError:
    print("⚠️  nvidia-ml-py не установлен, но это не критично")


# Импорты


In [ ]:
import os
import json
import glob
import random
import time
from pathlib import Path
from typing import List, Dict, Tuple
from collections import Counter, defaultdict
import re

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm

# Для работы с аудио
import librosa
import soundfile as sf

# NeMo
import nemo.collections.asr as nemo_asr
from nemo.core.config import hydra_runner
from nemo.utils import logging

# Для токенизации
from sentencepiece import SentencePieceTrainer, SentencePieceProcessor

# PyTorch
import torch

# Настройка для воспроизводимости
SEED = 42

# Устанавливаем seed для всех генераторов случайных чисел
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# Для воспроизводимости на GPU
if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Устанавливаем seed для Lightning через переменную окружения
os.environ['PL_GLOBAL_SEED'] = str(SEED)

print(f"Seed установлен: {SEED}")
print(f"PL_GLOBAL_SEED: {os.environ.get('PL_GLOBAL_SEED')}")


# Подготовка данных


In [ ]:
# Пути к датасетам
SOVA_PATH = "/share/audio_data/sova/ytub/raid/nanosemantics/nextcloud/sova_done"
SBER_GOLOS_PATH = "/share/audio_data/sber-golos/tar/train"

# Обрабатывать только указанные части из СОВА (для ускорения)
# Можно добавить больше частей: ['part_0', 'part_1', 'part_2', 'part_3', ...]
SOVA_PARTS = ['part_0', 'part_1', 'part_2']

# Выходные директории
OUTPUT_DIR = "./asr_data"
MANIFEST_DIR = os.path.join(OUTPUT_DIR, "manifests")
TOKENIZER_DIR = os.path.join(OUTPUT_DIR, "tokenizer")

os.makedirs(MANIFEST_DIR, exist_ok=True)
os.makedirs(TOKENIZER_DIR, exist_ok=True)


In [ ]:
def load_audio_info(audio_path):
    try:
        y, sr = librosa.load(audio_path, sr=None)
        duration = len(y) / sr
        return duration, sr
    except Exception as e:
        print(f"Ошибка при загрузке {audio_path}: {e}")
        return None, None

def normalize_text(text):
    if not text:
        return ""
    text = text.lower().strip()
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^а-яёa-z0-9\s]', '', text)
    text = text.strip()
    return text if len(text) > 0 else None

def process_sova_dataset(base_path, parts=None, max_files=None):
    manifest = []
    audio_extensions = ['.wav', '.mp3', '.flac', '.ogg']
    audio_files = []
    
    print("Поиск аудиофайлов в СОВА...")
    search_start = time.time()
    
    if parts:
        print(f"Обработка частей: {parts}")
        for part in parts:
            part_path = os.path.join(base_path, part)
            if not os.path.exists(part_path):
                print(f"Предупреждение: часть {part} не найдена")
                continue
            
            print(f"\nОбработка {part}...")
            part_start = time.time()
            
            # Собираем все поддиректории для оценки прогресса
            all_dirs = []
            print("  Сканирование структуры директорий...")
            for root, dirs, files in os.walk(part_path):
                all_dirs.append(root)
            
            print(f"  Найдено {len(all_dirs)} поддиректорий")
            
            # Поиск файлов с прогресс-баром
            part_files = []
            ext_counts = {ext: 0 for ext in audio_extensions}
            
            pbar_search = tqdm(all_dirs, desc=f"Поиск в {part}", unit="дир", ncols=100, leave=False)
            for root in pbar_search:
                try:
                    for file in os.listdir(root):
                        file_path = os.path.join(root, file)
                        if os.path.isfile(file_path):
                            file_ext = os.path.splitext(file)[1].lower()
                            if file_ext in audio_extensions:
                                part_files.append(file_path)
                                ext_counts[file_ext] += 1
                                pbar_search.set_postfix({
                                    'найдено': len(part_files),
                                    'wav': ext_counts['.wav'],
                                    'mp3': ext_counts['.mp3']
                                })
                except (PermissionError, OSError) as e:
                    continue
            
            audio_files.extend(part_files)
            part_time = time.time() - part_start
            print(f"  {part}: найдено {len(part_files)} файлов за {part_time:.1f}с ({part_time/60:.1f} мин)")
    else:
        # Обрабатываем все части (старый метод для совместимости)
        for ext in ['*.wav', '*.mp3', '*.flac', '*.ogg']:
            audio_files.extend(glob.glob(os.path.join(base_path, '**', ext), recursive=True))
    
    if max_files:
        audio_files = audio_files[:max_files]
    
    search_time = time.time() - search_start
    print(f"\n{'='*60}")
    print(f"Поиск завершен: найдено {len(audio_files)} аудиофайлов")
    print(f"Время поиска: {search_time:.1f}с ({search_time/60:.1f} мин)")
    print(f"{'='*60}\n")
    
    print("Начинаю обработку файлов...")
    start_time = time.time()
    processed = 0
    skipped_no_text = 0
    skipped_duration = 0
    
    pbar = tqdm(audio_files, desc="Обработка СОВА", unit="файл", ncols=100)
    for audio_path in pbar:
        duration, sr = load_audio_info(audio_path)
        if duration is None:
            continue
        
        if duration < 0.5:
            skipped_duration += 1
            continue
        
        text_path = audio_path.rsplit('.', 1)[0] + '.txt'
        if not os.path.exists(text_path):
            skipped_no_text += 1
            continue
        
        with open(text_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()
        
        text = normalize_text(text)
        
        if text and len(text) > 0:
            manifest.append({
                "audio_filepath": audio_path,
                "duration": duration,
                "text": text
            })
            processed += 1
        
        elapsed = time.time() - start_time
        pbar.set_postfix({
            'Обработано': processed,
            'Пропущено': skipped_no_text + skipped_duration,
            'Время': f"{elapsed:.1f}с"
        })
    
    elapsed_total = time.time() - start_time
    print(f"\nОбработка завершена!")
    print(f"  Обработано записей: {len(manifest)}")
    print(f"  Пропущено (нет текста): {skipped_no_text}")
    print(f"  Пропущено (длительность): {skipped_duration}")
    print(f"  Время обработки: {elapsed_total:.1f} секунд ({elapsed_total/60:.1f} минут)")
    print(f"  Скорость: {len(audio_files)/elapsed_total:.1f} файлов/сек")
    
    return manifest

def process_sber_golos_dataset(base_path, max_files=None):
    manifest = []
    manifest_path = os.path.join(base_path, 'manifest.json')
    
    if os.path.exists(manifest_path):
        with open(manifest_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            
        print(f"Найдено {len(data)} записей в манифесте СберГолос")
        print("Начинаю обработку...")
        
        for item in tqdm(data, desc="Обработка СберГолос", unit="запись"):
            audio_path = item.get('audio_filepath') or item.get('audio')
            if not audio_path:
                continue
            
            if not os.path.isabs(audio_path):
                audio_path = os.path.join(base_path, audio_path)
            
            duration, sr = load_audio_info(audio_path)
            if duration is None:
                continue
            
            text = item.get('text') or item.get('transcript', '')
            text = normalize_text(text)
            
            if text and len(text) > 0 and duration > 0.5:
                manifest.append({
                    "audio_filepath": audio_path,
                    "duration": duration,
                    "text": text
                })
        
        print(f"Обработано {len(manifest)} записей из {len(data)} в манифесте")
    else:
        audio_extensions = ['*.wav', '*.mp3', '*.flac', '*.ogg']
        audio_files = []
        for ext in audio_extensions:
            audio_files.extend(glob.glob(os.path.join(base_path, '**', ext), recursive=True))
        
        if max_files:
            audio_files = audio_files[:max_files]
        
        print(f"Найдено {len(audio_files)} аудиофайлов в СберГолос")
        print("Начинаю обработку...")
        
        for audio_path in tqdm(audio_files, desc="Обработка СберГолос", unit="файл"):
            duration, sr = load_audio_info(audio_path)
            if duration is None:
                continue
            
            text_path = audio_path.rsplit('.', 1)[0] + '.txt'
            if os.path.exists(text_path):
                with open(text_path, 'r', encoding='utf-8') as f:
                    text = f.read().strip()
                text = normalize_text(text)
                
                if text and len(text) > 0 and duration > 0.5:
                    manifest.append({
                        "audio_filepath": audio_path,
                        "duration": duration,
                        "text": text
                    })
        
        print(f"Обработано {len(manifest)} записей из {len(audio_files)} файлов")
    
    return manifest


In [ ]:
print("=" * 60)
print("ПОДГОТОВКА ДАННЫХ")
print("=" * 60)

print("\nЗагрузка данных из СОВА...")
sova_manifest = process_sova_dataset(SOVA_PATH, parts=SOVA_PARTS)

print("\n" + "=" * 60)
print("Загрузка данных из СберГолос...")
print("=" * 60)
sber_manifest = process_sber_golos_dataset(SBER_GOLOS_PATH)

all_manifest = sova_manifest + sber_manifest

print(f"\n{'='*60}")
print(f"ИТОГИ ЗАГРУЗКИ ДАННЫХ")
print(f"{'='*60}")
print(f"Всего записей: {len(all_manifest)}")
print(f"Из СОВА: {len(sova_manifest)} ({100*len(sova_manifest)/len(all_manifest):.1f}%)")
print(f"Из СберГолос: {len(sber_manifest)} ({100*len(sber_manifest)/len(all_manifest):.1f}%)")

print("\nВычисление статистики по длительности...")
durations = [item['duration'] for item in tqdm(all_manifest, desc="Обработка длительностей", unit="запись", leave=False)]

print(f"\nСтатистика по длительности:")
print(f"  Минимум: {min(durations):.2f} сек")
print(f"  Максимум: {max(durations):.2f} сек")
print(f"  Среднее: {np.mean(durations):.2f} сек")
print(f"  Медиана: {np.median(durations):.2f} сек")
print(f"  Общая длительность: {sum(durations)/3600:.2f} часов")

print(f"\n{'='*60}")
print("ЗАГРУЗКА ДАННЫХ ЗАВЕРШЕНА")
print(f"{'='*60}")


In [ ]:
print("=" * 60)
print("РАЗДЕЛЕНИЕ ДАННЫХ")
print("=" * 60)

print("Перемешивание данных...")
random.shuffle(all_manifest)

train_size = int(0.8 * len(all_manifest))
val_size = int(0.1 * len(all_manifest))

train_manifest = all_manifest[:train_size]
val_manifest = all_manifest[train_size:train_size + val_size]
test_manifest = all_manifest[train_size + val_size:]

print(f"\nРазделение данных:")
print(f"  Train: {len(train_manifest)} записей ({100*len(train_manifest)/len(all_manifest):.1f}%)")
print(f"  Val: {len(val_manifest)} записей ({100*len(val_manifest)/len(all_manifest):.1f}%)")
print(f"  Test: {len(test_manifest)} записей ({100*len(test_manifest)/len(all_manifest):.1f}%)")

def save_manifest(manifest, path):
    with open(path, 'w', encoding='utf-8') as f:
        for item in tqdm(manifest, desc=f"Сохранение {os.path.basename(path)}", unit="запись", leave=False):
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

print("\nСохранение манифестов...")
save_manifest(train_manifest, os.path.join(MANIFEST_DIR, 'train_manifest.json'))
save_manifest(val_manifest, os.path.join(MANIFEST_DIR, 'val_manifest.json'))
save_manifest(test_manifest, os.path.join(MANIFEST_DIR, 'test_manifest.json'))

print(f"\n{'='*60}")
print("Манифесты сохранены!")
print(f"{'='*60}")


# Обучение токенизатора


In [ ]:
all_texts = [item['text'] for item in train_manifest]

text_file = os.path.join(TOKENIZER_DIR, 'train_texts.txt')
with open(text_file, 'w', encoding='utf-8') as f:
    for text in all_texts:
        f.write(text + '\n')

print(f"Собрано {len(all_texts)} текстов для обучения токенизатора")
print(f"Общая длина текста: {sum(len(t) for t in all_texts)} символов")


In [ ]:
VOCAB_SIZE = 369

tokenizer_model = os.path.join(TOKENIZER_DIR, 'tokenizer.model')

SentencePieceTrainer.train(
    input=text_file,
    model_prefix=tokenizer_model.replace('.model', ''),
    vocab_size=VOCAB_SIZE,
    model_type='bpe',
    character_coverage=0.9995,
    max_sentencepiece_length=16,
    split_by_whitespace=True,
    byte_fallback=True,
    normalization_rule_name='nmt_nfkc_cf',
    user_defined_symbols=['<pad>', '<unk>', '<s>', '</s>']
)

print(f"Токенизатор обучен! Размер словаря: {VOCAB_SIZE}")


In [ ]:
tokenizer = SentencePieceProcessor()
tokenizer.load(tokenizer_model)

print(f"Размер словаря токенизатора: {tokenizer.get_piece_size()}")

test_texts = [
    "привет как дела",
    "распознавание речи это интересно",
    all_texts[0] if all_texts else "тестовый текст"
]

for text in test_texts:
    tokens = tokenizer.encode(text, out_type=str)
    decoded = tokenizer.decode(tokenizer.encode(text))
    print(f"\nТекст: {text}")
    print(f"Токены ({len(tokens)}): {tokens[:10]}..." if len(tokens) > 10 else f"Токены ({len(tokens)}): {tokens}")
    print(f"Декодировано: {decoded}")


# Обучение модели


In [ ]:
try:
    asr_model = nemo_asr.models.EncDecCTCModel.from_pretrained(
        model_name="QuartzNet15x5Base-En"
    )
    print("Загружена модель QuartzNet15x5Base-En")
except Exception as e:
    print(f"Не удалось загрузить предобученную модель: {e}")


In [ ]:
asr_model._cfg.train_ds.manifest_filepath = os.path.join(MANIFEST_DIR, 'train_manifest.json')
asr_model._cfg.validation_ds.manifest_filepath = os.path.join(MANIFEST_DIR, 'val_manifest.json')

asr_model._cfg.train_ds.sample_rate = 16000
asr_model._cfg.validation_ds.sample_rate = 16000
asr_model._cfg.train_ds.batch_size = 16
asr_model._cfg.validation_ds.batch_size = 16

print("Конфигурация модели обновлена")
print(f"Train manifest: {asr_model._cfg.train_ds.manifest_filepath}")
print(f"Val manifest: {asr_model._cfg.validation_ds.manifest_filepath}")


# Настройка Trainer


In [ ]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

checkpoint_dir = os.path.join(OUTPUT_DIR, 'checkpoints')
os.makedirs(checkpoint_dir, exist_ok=True)

logger = TensorBoardLogger(
    save_dir=OUTPUT_DIR,
    name='asr_training'
)

checkpoint_callback = ModelCheckpoint(
    dirpath=checkpoint_dir,
    filename='asr-{epoch:02d}-{val_loss:.2f}',
    monitor='val_loss',
    mode='min',
    save_top_k=3,
    save_last=True
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    mode='min',
    patience=5,
    verbose=True
)

trainer = Trainer(
    max_epochs=50,
    accelerator='gpu',
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback, early_stopping],
    log_every_n_steps=10,
    val_check_interval=0.5,
    gradient_clip_val=1.0,
    accumulate_grad_batches=1,
    deterministic=True,
    benchmark=False
)

print("Trainer настроен")
print(f"Seed: {SEED}")
print(f"Deterministic: {trainer.deterministic}")
print(f"Benchmark: {trainer.benchmark}")


# Запуск обучения


In [ ]:
trainer.fit(asr_model)

asr_model.save_to("./asr_model.nemo")


# Анализ результатов


In [ ]:
asr_model = nemo_asr.models.EncDecCTCModel.restore_from(
    restore_path="./asr_model.nemo"
)


# Визуализация метрик


In [ ]:
def plot_training_metrics(log_dir):
    try:
        from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
        
        # Ищем логи в поддиректориях (lightning_logs/version_*/)
        version_dirs = []
        if os.path.exists(log_dir):
            for item in os.listdir(log_dir):
                item_path = os.path.join(log_dir, item)
                if os.path.isdir(item_path) and (item.startswith('version_') or 'events.out.tfevents' in os.listdir(item_path) if os.path.isdir(item_path) else False):
                    version_dirs.append(item_path)
        
        if not version_dirs:
            # Пробуем использовать log_dir напрямую
            version_dirs = [log_dir]
        
        latest_version = max(version_dirs) if version_dirs else log_dir
        event_dir = latest_version
        
        event_acc = EventAccumulator(event_dir)
        event_acc.Reload()
        
        scalar_tags = event_acc.Tags()['scalars']
        
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))
        
        # Loss
        if 'train/loss' in scalar_tags or 'train_loss' in scalar_tags:
            tag = 'train/loss' if 'train/loss' in scalar_tags else 'train_loss'
            train_loss = event_acc.Scalars(tag)
            axes[0, 0].plot([s.step for s in train_loss], [s.value for s in train_loss], label='Train Loss')
        
        if 'val/loss' in scalar_tags or 'val_loss' in scalar_tags:
            tag = 'val/loss' if 'val/loss' in scalar_tags else 'val_loss'
            val_loss = event_acc.Scalars(tag)
            axes[0, 0].plot([s.step for s in val_loss], [s.value for s in val_loss], label='Val Loss')
        
        axes[0, 0].set_xlabel('Step')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Training and Validation Loss')
        axes[0, 0].legend()
        axes[0, 0].grid(True)
        
        # WER
        if 'val/wer' in scalar_tags or 'val_wer' in scalar_tags:
            tag = 'val/wer' if 'val/wer' in scalar_tags else 'val_wer'
            val_wer = event_acc.Scalars(tag)
            axes[0, 1].plot([s.step for s in val_wer], [s.value for s in val_wer], label='Val WER', color='green')
            axes[0, 1].set_xlabel('Step')
            axes[0, 1].set_ylabel('WER')
            axes[0, 1].set_title('Word Error Rate')
            axes[0, 1].legend()
            axes[0, 1].grid(True)
        
        # Learning Rate (если есть)
        if 'train/learning_rate' in scalar_tags or 'learning_rate' in scalar_tags:
            tag = 'train/learning_rate' if 'train/learning_rate' in scalar_tags else 'learning_rate'
            lr = event_acc.Scalars(tag)
            axes[1, 0].plot([s.step for s in lr], [s.value for s in lr], label='Learning Rate', color='orange')
            axes[1, 0].set_xlabel('Step')
            axes[1, 0].set_ylabel('LR')
            axes[1, 0].set_title('Learning Rate')
            axes[1, 0].legend()
            axes[1, 0].grid(True)
            axes[1, 0].set_yscale('log')
        
        plt.tight_layout()
        plt.savefig(os.path.join(OUTPUT_DIR, 'training_metrics.png'), dpi=300)
        plt.show()
        
    except Exception as e:
        print(f"Ошибка при загрузке метрик: {e}")
        import traceback
        traceback.print_exc()

plot_training_metrics(logger.log_dir)


# Тестирование модели


In [ ]:
def test_model(model, test_manifest, num_samples=10):
    samples = random.sample(test_manifest, min(num_samples, len(test_manifest)))
    results = []
    
    for sample in samples:
        audio_path = sample['audio_filepath']
        true_text = sample['text']
        
        try:
            predicted_text = model.transcribe([audio_path])[0]
            
            results.append({
                'audio': audio_path,
                'true': true_text,
                'predicted': predicted_text
            })
            
            print(f"\nАудио: {os.path.basename(audio_path)}")
            print(f"Истинный текст: {true_text}")
            print(f"Распознанный текст: {predicted_text}")
            
        except Exception as e:
            print(f"Ошибка при обработке {audio_path}: {e}")
    
    return results

test_results = test_model(asr_model, test_manifest, num_samples=20)


# Вычисление метрик качества


In [ ]:
def calculate_wer(true_text, predicted_text):
    try:
        from jiwer import wer
        return wer(true_text, predicted_text)
    except:
        true_words = true_text.lower().split()
        pred_words = predicted_text.lower().split()
        
        if len(true_words) == 0:
            return 1.0 if len(pred_words) > 0 else 0.0
        
        errors = sum(1 for t, p in zip(true_words, pred_words) if t != p)
        errors += abs(len(true_words) - len(pred_words))
        return errors / len(true_words)

def calculate_metrics(model, test_manifest):
    wers = []
    errors = 0
    
    print(f"Вычисление метрик на {len(test_manifest)} примерах...")
    
    for sample in tqdm(test_manifest, desc="Оценка качества", unit="пример", ncols=100):
        audio_path = sample['audio_filepath']
        true_text = sample['text']
        
        try:
            predicted_text = model.transcribe([audio_path])[0]
            wer_score = calculate_wer(true_text, predicted_text)
            wers.append(wer_score)
        except Exception as e:
            errors += 1
            if errors <= 5:  # Показываем только первые 5 ошибок
                print(f"\nОшибка при обработке {os.path.basename(audio_path)}: {e}")
    
    if not wers:
        print("✗ Не удалось обработать ни одного примера")
        return {
            'avg_wer': 1.0,
            'median_wer': 1.0,
            'min_wer': 1.0,
            'max_wer': 1.0,
            'all_wers': []
        }
    
    avg_wer = np.mean(wers)
    
    print(f"\n{'='*60}")
    print(f"РЕЗУЛЬТАТЫ ОЦЕНКИ")
    print(f"{'='*60}")
    print(f"Обработано успешно: {len(wers)}/{len(test_manifest)}")
    if errors > 0:
        print(f"Ошибок: {errors}")
    print(f"\nМетрики WER:")
    print(f"  Средний WER: {avg_wer:.4f}")
    print(f"  Медианный WER: {np.median(wers):.4f}")
    print(f"  Минимальный WER: {np.min(wers):.4f}")
    print(f"  Максимальный WER: {np.max(wers):.4f}")
    print(f"  Стандартное отклонение: {np.std(wers):.4f}")
    print(f"{'='*60}")
    
    return {
        'avg_wer': avg_wer,
        'median_wer': np.median(wers),
        'min_wer': np.min(wers),
        'max_wer': np.max(wers),
        'std_wer': np.std(wers),
        'all_wers': wers,
        'errors': errors
    }

metrics = calculate_metrics(asr_model, test_manifest)
